In [144]:
import geocoder
import unicodecsv
import logging
import time
import csv
from pygeocoder import Geocoder
from uszipcode import ZipcodeSearchEngine
import pandas as pd
import blaze
import time
import os


/anaconda3/lib/python3.6/site-packages/odo/backends/pandas.py:102: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access NaTType as type(pandas.NaT)
  @convert.register((pd.Timestamp, pd.Timedelta), (pd.tslib.NaTType, type(None)))


## Functions

In [145]:
def drop_columns_with_suffix(df):
    # list comprehension of the cols that end with '_y'
    to_drop = [x for x in df if x.endswith('_y')]
    df.drop(to_drop, axis=1, inplace=True)
    to_drop = [x for x in df if x.endswith('_x')]
    df.drop(to_drop, axis=1, inplace=True)

## Map Lat and Long to  Zipcodes

In [39]:
#https://pythonhosted.org/uszipcode/
with open('Social_Economic_Info_By_Zipcode/latlongzip.csv', 'w') as csvfile:
                fieldnames = ['lat', 'long', 'zipcode']
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writeheader()

with open('Social_Economic_Info_By_Zipcode/latlong.csv', 'r') as f:
    reader = csv.DictReader(f)
    for line in reader: 
          lat = float(line['lat'])
          lon = float(line['long'])
          res = search.by_coordinate(lat, lon, returns=1)
          with open('Social_Economic_Info_By_Zipcode/latlongzip.csv', 'a') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writerow({'lat': lat, 'long': lon, 'zipcode': res[0]["Zipcode"] })
        
#search = ZipcodeSearchEngine()
#zipcode = search.by_zipcode("60613")
#print(res[0]["Zipcode"])

## Map Census Group, Zipcode, GeoID

In [22]:
#DATA SOURCE : http://looker-datablocks.s3-website-us-east-1.amazonaws.com/
geoid_group_df = pd.read_csv('Social_Economic_Info_By_Zipcode/Geoid_group_mapping.csv',header=0)
geoid_zipcode_df = pd.read_csv('Social_Economic_Info_By_Zipcode/Geoid_zipcode_mapping.csv',header=0)
facts_df = pd.read_csv('Social_Economic_Info_By_Zipcode/fast_facts.csv',header=0)

geoid_group_zip_df = geoid_group_df.merge(geoid_zipcode_df,left_on=['geoid11'], right_on=['GEOID'],how='inner')
demographics_df = geoid_group_zip_df.merge(facts_df,left_on=['blkgrp_map_geoid'], right_on=['logrecno_bg_map_block_group'],how='inner')
demographics_df.to_csv('Social_Economic_Info_By_Zipcode/demographics.csv', sep=',')


## Read the Main Divvy Dataset

In [40]:
#parse_dates = true to read dates automatically
divvy_dataset_raw =pd.read_csv('Divvy/trip_id_weather/chicago-divvy-bicycle-sharing-data/data_raw.csv',parse_dates=True)

#Ensure the dataframe has no duplicates
divvy_dataset_raw = divvy_dataset_raw.drop_duplicates()

#Lets work with 2015, 2016 datasets
#divvy_dataset_raw = divvy_dataset_raw.loc[('year'== 2015) & ('year' == 2016),:]


## Merge zipcode data to the main dataset

In [41]:
divvy_dataset = divvy_dataset_raw.copy()

zipcode_df = pd.read_csv('Social_Economic_Info_By_Zipcode/latlongzip.csv',header=0)

#Populate start station zipcode
divvy_dataset=divvy_dataset.merge(zipcode_df,left_on=['latitude_start','longitude_start'], right_on=['lat','long'],how='left')
divvy_dataset.rename(inplace=True, columns={"zipcode": "zipcode_start"})

#Populate end station zipcode
divvy_dataset=divvy_dataset.merge(zipcode_df,left_on=['latitude_end','longitude_end'], right_on=['lat','long'],how='left')
divvy_dataset.rename(inplace=True, columns={"zipcode": "zipcode_end"})

#Convert zipcodes to integer
divvy_dataset['zipcode_start'].fillna(0).astype(int)
divvy_dataset['zipcode_end'].fillna(0).astype(int)

divvy_dataset = divvy_dataset.drop_duplicates()
drop_columns_with_suffix(divvy_dataset)

divvy_dataset.to_csv('Social_Economic_Info_By_Zipcode/MergedData/divvy_raw_zipcode.csv', sep=',')


## Distance between start and end station in meters

In [42]:
distance_df = pd.read_csv('Social_Economic_Info_By_Zipcode/Bike_route_distance_between_stations.csv',header=0)
divvy_dataset = divvy_dataset.merge(distance_df,left_on=['from_station_name','to_station_name'], right_on=['start_station','end_station'],how='left')
divvy_dataset.rename(inplace=True, columns={"distance": "distance_in_meters"})

distance_df = pd.read_csv('Social_Economic_Info_By_Zipcode/station_distance.csv',header=0)
divvy_dataset = divvy_dataset.merge(distance_df,left_on=['latitude_start','longitude_start','latitude_end','longitude_end'], right_on=['station_start_latitude','station_start_longitude','station_end_latitude','station_end_longitude'],how='left')
divvy_dataset.rename(inplace=True, columns={"distance_meters": "distance_meters_by_coord"})

#distance_df = pd.read_csv('Social_Economic_Info_By_Zipcode/station_distance.csv',header=0)
#divvy_dataset = divvy_dataset.merge(distance_df,left_on=['from_station_id','to_station_id'], right_on=['station_start','station_end'],how='inner')
#divvy_dataset.rename(inplace=True, columns={"distance_meters": "distance_meters_by_station"})

divvy_dataset["distance_in_meters"].fillna(0).astype(int)
divvy_dataset["distance_meters_by_coord"].fillna(0).astype(int)
#divvy_dataset["distance_meters_by_station"].fillna(0).astype(int)

#print('distance_meters_by_coord',divvy_dataset.groupby(divvy_dataset.distance_meters_by_coord).size())
#print('distance_meters_by_tation',divvy_dataset.groupby(divvy_dataset.distance_meters_by_station).size())

divvy_dataset.loc[divvy_dataset.distance_in_meters == 0.0,['distance_in_meters']] = divvy_dataset.distance_meters_by_coord

divvy_dataset.drop_duplicates()
drop_columns_with_suffix(divvy_dataset)
divvy_dataset.to_csv('Social_Economic_Info_By_Zipcode/MergedData/divvy_raw_zipcode_stationdistance.csv', sep=',')

print('distance_in_meters',divvy_dataset.distance_in_meters.value_counts())


distance_in_meters 0.0        126949
1455.0      25180
867.0       20318
1830.0      18780
1659.0      16797
1359.0      15381
3093.0      14625
904.0       14148
1159.0      14117
3023.0      13750
4489.0      13103
2488.0      13091
3603.0      12697
1108.0      12079
653.0       11647
1873.0      11457
4969.0      11450
949.0       11369
1172.0      11117
1485.0      10999
1786.0      10892
1366.0      10520
488.0       10457
3304.0      10417
1854.0      10170
4714.0       9913
760.0        9848
1112.0       9813
2939.0       9713
1051.0       9493
            ...  
9505.0          1
14589.0         1
14582.0         1
9495.0          1
11972.0         1
9493.0          1
11986.0         1
12010.0         1
12015.0         1
12016.0         1
14624.0         1
11945.0         1
11895.0         1
14636.0         1
14707.0         1
11897.0         1
9564.0          1
14701.0         1
11903.0         1
14694.0         1
14689.0         1
14687.0         1
14686.0         1
14685.0  

##  Social and Economic attributes for 2014, 2015, 2016

In [171]:
import os
#os.chdir('Social_Economic_Info_By_Zipcode/ACS/2014')
print("Path at terminal when executing this file")
print(os.getcwd() + "\n")

Path at terminal when executing this file
/Users/chetana/Documents/Python/Notebook/SpringBoard/Capstone1/Social_Economic_Info_By_Zipcode/ACS/2016



In [175]:
os.chdir('../2014')

In [176]:
#2014 Data
Mapping_2014_df = pd.read_csv('Geoid_group_mapping.csv',header=0,delimiter=',')
df1 = pd.read_csv('1.csv',header=0,delimiter=',')
df2 = pd.read_csv('2.csv',header=0,delimiter=',')
df3 = pd.read_csv('3.csv',header=0,delimiter=',')
df4 = pd.read_csv('4.csv',header=0,delimiter=',')
df5 = pd.read_csv('5.csv',header=0,delimiter=',')
df6 = pd.read_csv('6.csv',header=0,delimiter=',')
df7 = pd.read_csv('7.csv',header=0,delimiter=',')

dem_inc_pop_race_trp_2014=Mapping_2014_df.merge(df1,left_on=['blkgrp_map_geoid'], right_on=['Id2'],how='inner')
dem_inc_pop_race_trp_2014 = dem_inc_pop_race_trp_2014.merge(df2,left_on='blkgrp_map_geoid',right_on='Id2',how='inner')
dem_inc_pop_race_trp_2014 = dem_inc_pop_race_trp_2014.merge(df3,left_on='blkgrp_map_geoid',right_on='Id2',how='inner')
dem_inc_pop_race_trp_2014 = dem_inc_pop_race_trp_2014.merge(df4,left_on='blkgrp_map_geoid',right_on='Id2',how='inner')
dem_inc_pop_race_trp_2014 = dem_inc_pop_race_trp_2014.merge(df5,left_on='blkgrp_map_geoid',right_on='Id2',how='inner')
dem_inc_pop_race_trp_2014 = dem_inc_pop_race_trp_2014.merge(df6,left_on='blkgrp_map_geoid',right_on='Id2',how='inner')
dem_inc_pop_race_trp_2014 = dem_inc_pop_race_trp_2014.merge(df7,left_on='blkgrp_map_geoid',right_on='Id2',how='inner')
dem_inc_pop_race_trp_2014['year'] = 2014

dem_inc_pop_race_trp_2014.to_csv('../../../Social_Economic_Info_By_Zipcode/MergedData/dem_inc_pop_race_trp_2014.csv', sep=',')


In [173]:
os.chdir('../2015')

In [174]:
#2015 Data
Mapping_2015_df = pd.read_csv('Geoid_group_mapping.csv',header=0,delimiter=',')
df1 = pd.read_csv('1.csv',header=0,delimiter=',')
df2 = pd.read_csv('2.csv',header=0,delimiter=',')
df3 = pd.read_csv('3.csv',header=0,delimiter=',')
df4 = pd.read_csv('4.csv',header=0,delimiter=',')
df5 = pd.read_csv('5.csv',header=0,delimiter=',')
df6 = pd.read_csv('6.csv',header=0,delimiter=',')
df7 = pd.read_csv('7.csv',header=0,delimiter=',')

dem_inc_pop_race_trp_2015=Mapping_2015_df.merge(df1,left_on=['blkgrp_map_geoid'], right_on=['Id2'],how='inner')
dem_inc_pop_race_trp_2015 = dem_inc_pop_race_trp_2015.merge(df2,left_on='blkgrp_map_geoid',right_on='Id2',how='inner')
dem_inc_pop_race_trp_2015 = dem_inc_pop_race_trp_2015.merge(df3,left_on='blkgrp_map_geoid',right_on='Id2',how='inner')
dem_inc_pop_race_trp_2015 = dem_inc_pop_race_trp_2015.merge(df4,left_on='blkgrp_map_geoid',right_on='Id2',how='inner')
dem_inc_pop_race_trp_2015 = dem_inc_pop_race_trp_2015.merge(df5,left_on='blkgrp_map_geoid',right_on='Id2',how='inner')
dem_inc_pop_race_trp_2015 = dem_inc_pop_race_trp_2015.merge(df6,left_on='blkgrp_map_geoid',right_on='Id2',how='inner')
dem_inc_pop_race_trp_2015 = dem_inc_pop_race_trp_2015.merge(df7,left_on='blkgrp_map_geoid',right_on='Id2',how='inner')
dem_inc_pop_race_trp_2015['year'] = 2015

dem_inc_pop_race_trp_2015.to_csv('../../../Social_Economic_Info_By_Zipcode/MergedData/dem_inc_pop_race_trp_2015.csv', sep=',')


In [169]:
os.chdir('../2016')

In [172]:
#2016 Data
Mapping_2016_df = pd.read_csv('Geoid_group_mapping.csv',header=0,delimiter=',')
df1 = pd.read_csv('1.csv',header=0,delimiter=',')
df2 = pd.read_csv('2.csv',header=0,delimiter=',')
df3 = pd.read_csv('3.csv',header=0,delimiter=',')
df4 = pd.read_csv('4.csv',header=0,delimiter=',')
df5 = pd.read_csv('5.csv',header=0,delimiter=',')
df6 = pd.read_csv('6.csv',header=0,delimiter=',')
df7 = pd.read_csv('7.csv',header=0,delimiter=',')

dem_inc_pop_race_trp_2016=Mapping_2016_df.merge(df1,left_on=['blkgrp_map_geoid'], right_on=['Id2'],how='inner')
dem_inc_pop_race_trp_2016 = dem_inc_pop_race_trp_2016.merge(df2,left_on='blkgrp_map_geoid',right_on='Id2',how='inner')
dem_inc_pop_race_trp_2016 = dem_inc_pop_race_trp_2016.merge(df3,left_on='blkgrp_map_geoid',right_on='Id2',how='inner')
dem_inc_pop_race_trp_2016 = dem_inc_pop_race_trp_2016.merge(df4,left_on='blkgrp_map_geoid',right_on='Id2',how='inner')
dem_inc_pop_race_trp_2016 = dem_inc_pop_race_trp_2016.merge(df5,left_on='blkgrp_map_geoid',right_on='Id2',how='inner')
dem_inc_pop_race_trp_2016 = dem_inc_pop_race_trp_2016.merge(df6,left_on='blkgrp_map_geoid',right_on='Id2',how='inner')
dem_inc_pop_race_trp_2016 = dem_inc_pop_race_trp_2016.merge(df7,left_on='blkgrp_map_geoid',right_on='Id2',how='inner')
dem_inc_pop_race_trp_2016['year'] = 2016

dem_inc_pop_race_trp_2016.to_csv('../../../Social_Economic_Info_By_Zipcode/MergedData/dem_inc_pop_race_trp_2016.csv', sep=',')


## Add Date Attributes to the dataset

In [148]:
divvy_dataset = pd.read_csv('Social_Economic_Info_By_Zipcode/MergedData/divvy_raw_zipcode_stationdistance.csv', sep=',')

divvy_dataset.stoptime=pd.to_datetime(divvy_dataset.stoptime)
divvy_dataset.starttime=pd.to_datetime(divvy_dataset.starttime)
divvy_dataset['Year'] = divvy_dataset['starttime'].apply(lambda d: d.year)
divvy_dataset['Month'] = divvy_dataset['starttime'].apply(lambda d: d.month)
divvy_dataset['Day'] = divvy_dataset['starttime'].apply(lambda d: d.day)
divvy_dataset['DayOfWeek'] = divvy_dataset['starttime'].apply(lambda d: d.dayofweek)
divvy_dataset['DayName'] = divvy_dataset['starttime'].apply(lambda d: d.weekday_name)
divvy_dataset['DayOfYear'] = divvy_dataset['starttime'].apply(lambda d: d.dayofyear)
divvy_dataset['WeekOfYear'] = divvy_dataset['starttime'].apply(lambda d: d.weekofyear)
divvy_dataset['Quarter'] = divvy_dataset['starttime'].apply(lambda d: d.quarter)

divvy_dataset.to_csv('Social_Economic_Info_By_Zipcode/MergedData/divvy_raw_zipcode_stationdistance.csv', sep=',')


## Add zipcode to social and economic factors

In [7]:
import os
os.getcwd()

'/Users/chetana/Documents/Python/Notebook/SpringBoard/Capstone1'

In [33]:
soc_eco_df = pd.read_csv('Social_Economic_Info_By_Zipcode/MergedData/SourceData/social_economic_attributes.csv', header=0,sep=',')
zipcode_df = pd.read_csv('Social_Economic_Info_By_Zipcode/MergedData/SourceData/Geoid_zipcode_mapping.csv', header=0,sep=',')
school_df = pd.read_csv('Social_Economic_Info_By_Zipcode/MergedData/SourceData/school_information.csv', header=0,sep=',')

soc_eco_df.columns = soc_eco_df.columns.str.strip()
zipcode_df.columns = zipcode_df.columns.str.strip()
school_df.columns = school_df.columns.str.strip()
school_df['school_zip_code'] = school_df['school_zip_code'].fillna(0).astype(int)

soc_eco_df = soc_eco_df.merge(zipcode_df,left_on='geoid11',right_on='geoid',how='left')
soc_eco_df.drop_duplicates()
soc_eco_df = soc_eco_df.merge(school_df,left_on='zipcode',right_on='school_zip_code',how='outer')
soc_eco_df.drop_duplicates()

soc_eco_df['zipcode'] = soc_eco_df['zipcode'].fillna(0).astype(int)
soc_eco_df.to_csv('Social_Economic_Info_By_Zipcode/MergedData/social_economic_zipcode_attributes.csv', sep=',')

soc_eco_df.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 556 entries, 0 to 555
Data columns (total 60 columns):
logrecno                                   470 non-null float64
tract_x                                    470 non-null float64
blkgrp                                     470 non-null float64
geoid11                                    470 non-null float64
blkgrp_map_geoid                           470 non-null float64
tract_name                                 470 non-null object
block_group_name                           470 non-null object
latitude                                   470 non-null float64
longitude                                  470 non-null float64
square_miles_land                          470 non-null float64
square_miles_water                         470 non-null float64
year                                       470 non-null float64
total_population                           470 non-null float64
median_household_income                    470 non-null object
pe

In [143]:
#You need a dictionary that maps each key to multiple values.
csvfile='Social_Economic_Info_By_Zipcode/MergedData/SourceData/zipcode_multiple_geoids.csv'
dict_to_csvfile='Social_Economic_Info_By_Zipcode/MergedData/SourceData/dict_zipcode_multiple_geoids.csv'
dict_transpose_to_csvfile='Social_Economic_Info_By_Zipcode/MergedData/SourceData/dict_transpose_zipcode_multiple_geoids.csv'

d1 = {}
with open(csvfile, mode='r') as infile:
    reader = csv.reader(infile)
    next(reader)
    for row in reader:
        d1.setdefault(row[0], []).append(row[1])

with open(dict_to_csvfile, 'w') as f:  # Just use 'w' mode in 3.x
    w = csv.DictWriter(f, d1.keys())
    w.writeheader()
    w.writerow(d1)

## Transpose rows to columns (not tested)    
#from more_itertools import izip
#a = izip(*csv.reader(open(dict_to_csvfile, "r")))
#csv.writer(open(dict_transpose_to_csvfile, "wb")).writerows(a)

# Requires array of same length
#pd.DataFrame.from_dict(d1)

geoid_zip_map = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in d1.items() ]))

x = list(geoid_zip_map['60601'].dropna())


NameError: name 'csv' is not defined

## Load Divvy and Social Economic Datasets into a dataframe

In [ ]:
divvy_df = pd.read_csv('Social_Economic_Info_By_Zipcode/MergedData/divvy_raw_zipcode_stationdistance.csv', sep=',')
soc_eco_df = pd.read_csv('Social_Economic_Info_By_Zipcode/MergedData/social_economic_zipcode_attributes.csv', sep=',')

#Drop Duplicates
divvy_df.drop_duplicates()

#Create a copy to maintain the original
divvy_orig_df = divvy_df.copy()

#Changes in the file
divvy_df['zipcode_start'] = divvy_df['zipcode_start'].fillna(0).astype(int)
divvy_df['zipcode_end'] = divvy_df['zipcode_end'].fillna(0).astype(int)

In [ ]:
#Check for missing values for Year Column
divvy_df.loc[divvy_df.Year.isnull(),:]
divvy_df.Year.isnull().values.any()
#####False

## Keep 2014, 2015, 2016 year data


In [ ]:
zipcode_df = pd.read_csv('Social_Economic_Info_By_Zipcode/MergedData/SourceData/transpose_dict_zipcode_multiple_geoids.csv', header=0,sep=';')
zipcode_df['zipcode'] = zipcode_df['zipcode'].fillna(0).astype(int)
divvy_df['zipcode_start'] = divvy_df['zipcode_start'].fillna(0).astype(int)
divvy_df = divvy_df.merge(zipcode_df,left_on='zipcode_start',right_on='zipcode',how='left')

divvy_df_2013 = divvy_df.loc[divvy_df.Year == 2013,:]
divvy_df_2017 = divvy_df.loc[divvy_df.Year == 2017,:]
divvy_13_17_df = pd.concat([divvy_df_2013, divvy_df_2017])

#Divvy Data for 2014, 2015, 2016
divvy_df_2014 = divvy_df.loc[divvy_df.Year == 2014,:]
divvy_df_2015 = divvy_df.loc[divvy_df.Year == 2015,:]
divvy_df_2016 = divvy_df.loc[divvy_df.Year == 2016,:]

divvy_df.drop_duplicates()

#Remove data from dataframe
#pd.concat([divvy_df, divvy_13_17_df]).drop_duplicates(keep=False)


## Merge Divvy and Social Economic data

In [ ]:
#Social Economic Data for 2014, 2015, 2016
soc_eco_df = pd.read_csv('Social_Economic_Info_By_Zipcode/MergedData/social_economic_zipcode_attributes.csv', sep=',')
soc_eco_df['zipcode'] = soc_eco_df['zipcode'].fillna(0).astype(int)

soc_eco_df_2014 = soc_eco_df.loc[soc_eco_df.year == 2014,:]
soc_eco_df_2015 = soc_eco_df.loc[soc_eco_df.year == 2015,:]
soc_eco_df_2016 = soc_eco_df.loc[soc_eco_df.year == 2016,:]

soc_eco_df_2014.zipcode.isnull().values.any()

In [ ]:
#YEAR : No of Records : Size of File
#2014 : 3783945 : 2.8GB
#2015 : 4717366 : 3.32GB
#2016 : 5309852 : 3.62GB

#2014
divvy_soc_eco_2014=divvy_df_2014.merge(soc_eco_df_2014,left_on=['zipcode_start'], right_on=['zipcode'],how='left')
divvy_soc_eco_2014.drop(['geoid','logrecno','blkgrp','geoid11','blkgrp_map_geoid','latitude','longitude','block_group_name','tract_name','city','station_start','station_end','distance_meters_by_coord','station_start_latitude','station_start_longitude','station_end_latitude','station_end_longitude','start_station','end_station','station_start','station_end'],inplace=True,axis=1)
divvy_soc_eco_2014.drop_duplicates()
drop_columns_with_suffix(divvy_soc_eco_2014)
divvy_soc_eco_2014.to_csv('Social_Economic_Info_By_Zipcode/MergedData/Divvy_2014_merged.csv')

#2015
divvy_soc_eco_2015=divvy_df_2015.merge(soc_eco_df_2015,left_on=['zipcode_start'], right_on=['zipcode'],how='left')
divvy_soc_eco_2015.drop(['geoid','logrecno','blkgrp','geoid11','blkgrp_map_geoid','latitude','longitude','block_group_name','tract_name','city','station_start','station_end','distance_meters_by_coord','station_start_latitude','station_start_longitude','station_end_latitude','station_end_longitude','start_station','end_station','station_start','station_end'],inplace=True,axis=1)
divvy_soc_eco_2015.drop_duplicates()
drop_columns_with_suffix(divvy_soc_eco_2015)
divvy_soc_eco_2015.to_csv('Social_Economic_Info_By_Zipcode/MergedData/Divvy_2015_merged.csv')

#2016
divvy_soc_eco_2016=divvy_df_2016.merge(soc_eco_df_2016,left_on=['zipcode_start'], right_on=['zipcode'],how='left')
divvy_soc_eco_2016.drop(['geoid','logrecno','blkgrp','geoid11','blkgrp_map_geoid','latitude','longitude','block_group_name','tract_name','city','station_start','station_end','distance_meters_by_coord','station_start_latitude','station_start_longitude','station_end_latitude','station_end_longitude','start_station','end_station','station_start','station_end'],inplace=True,axis=1)
divvy_soc_eco_2016.drop_duplicates()
drop_columns_with_suffix(divvy_soc_eco_2016)
divvy_soc_eco_2016.to_csv('Social_Economic_Info_By_Zipcode/MergedData/Divvy_2016_merged.csv')

#divvy_soc_eco_2014.loc[divvy_soc_eco_2014.zipcode_start == 60601,:].to_csv('Social_Economic_Info_By_Zipcode/MergedData/Divvy_2014_merged_60601.csv')

#divvy_14_15_16_df = pd.concat([divvy_soc_eco_2014, divvy_soc_eco_2015, divvy_soc_eco_2016])
#divvy_14_15_16_df.to_csv('Social_Economic_Info_By_Zipcode/MergedData/Divvy_Social_Economic.csv')

#Get rows with NANs
#nans = lambda df: df[df.isnull().any(axis=1)]
#nans(divvy_soc_eco_2014)

In [ ]:
'trip_id'
,'usertype'
,'gender'
,'starttime'
,'stoptime'
,'tripduration'
,'from_station_id'
,'to_station_id'
,'from_station_name'
,'to_station_name'
,'dpcapacity_start'
,'latitude_start'
,'longitude_start'
,'latitude_end'
,'longitude_end'
,'dpcapacity_end'
,'temperature'
,'windchill'
,'dewpoint'
,'humidity'
,'pressure'
,'visibility'
,'wind_speed'
,'precipitation'
,'events'
,'rain'
,'conditions'
,'zipcode_start'
,'zipcode_end'
,'distance_in_meters'
,'Year'
,'Month'
,'Day'
,'DayOfWeek'
,'DayName'
,'DayOfYear'
,'WeekOfYear'
,'Quarter'
,'geoid_list'
,'square_miles_land'
,'square_miles_water'
,'year'
,'total_population'
,'median_household_income'
,'per_capita_income'
,'median_age_total'
,'median_age_male'
,'median_age_female'
,'race_total'
,'income_less_than_$10000'
,'income_$10000_to_$14999'
,'income_$15000_to_$19999'
,'income_$20000_to_$24999'
,'income_$25000_to_$29999'
,'income_$30000_to_$34999'
,'income_$35000_to_$39999'
,'income_$40000_to_$44999'
,'income_$45000_to_$49999'
,'income_$50000_to_$59999'
,'income_$60000_to_$74999'
,'income_$75000_to_$99999'
,'income_$100000_to_$124999'
,'income_$125000_to_$149999'
,'income_$150000_to_$199999'
,'income_$200000_or_more'
,'aggregate_travel_time_in_minutes'
,'one_person_carpool'
,'two_person_carpool'
,'three_person_carpool'
,'public_transportation'
,'Walked'
,'taxi_motorcycle_bike'
,'White_alone',
,'Black_or_African_American_alone'
,'American_Indian_and_Alaska_Native_alone'
,'Asian_alone'
,'some_other_race_alone'
,'two_or_more_races'
,'zpoppct'
,'school_score'
,'school_name'
,'school_address'
,'school_city'
,'school_zip_code'
,'school_enrollment'
,'school_grades'


## Add name of the month using integer values

In [ ]:
# parse_dates = true to read dates automatically
divvyTrips =pd.read_csv('Social_Economic_Info_By_Zipcode/MergedData/divvy_raw_zipcode_stationdistance.csv',parse_dates=True)

#Ensure the dataframe has no duplicates
divvyTrips = divvyTrips.drop_duplicates()
#Remove all rows with missing values. axis =0 is the default 
divvyTrips = divvyTrips.dropna(subset=['trip_id'], how="all", axis=0)

months = {1: 'Jan', 2: 'Feb', 3: 'Mar',4: 'Apr',5:'May',6: 'Jun',7:'Jul',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12: 'Dec'}

divvyTrips['month_name'] = divvyTrips['Month'].map(months)
divvyTrips['months-year'] = divvyTrips.reset_index().apply(lambda x: '{}-{}'.format(x['month_name'],x['Year']),axis=1)

#divvy_dataset['month-year'] = divvy_dataset.reset_index().apply(lambda x: '{}-{}'.format(x['months'],x['year']),axis=1)

#Convert Trip to minutes
divvyTrips['tripInMinutes'] = divvyTrips['tripduration'] / 60

#Add the Hour column
divvyTrips.starttime = pd.to_datetime(divvyTrips.starttime)
divvyTrips['Hour'] = divvyTrips.starttime.apply(lambda x: x.hour)

## If trip start time is greater than stop time, swap them
divvyTrips.loc[divvyTrips.starttime > divvyTrips.stoptime,['starttime','stoptime']] = divvyTrips.loc[divvyTrips.starttime > divvyTrips.stoptime,['stoptime','starttime']].values

#Drop columns 
divvyTrips.drop(['Unnamed: 0','Unnamed: 0.1'],inplace=True,axis=1)

#Remove Dependent user type
divvyTrips = divvyTrips[divvyTrips.usertype != 'Dependent']

# Drop Duplicates
divvyTrips = divvyTrips.drop_duplicates()

## Group Station capacity into buckets

In [ ]:
#Convert an existing Series or column to a category dtype:
divvyTrips['gender'].astype('category')

int(divvyTrips.dpcapacity_start.max())
##### 55

labels = ["{0} - {1}".format(i, i + 9) for i in range(int(divvyTrips.dpcapacity_start.min()), int(divvyTrips.dpcapacity_start.max())*2, 10)]

#divvy_dataset['group'] = pd.cut(divvy_dataset.dpcapacity_start, range(int(divvy_dataset.dpcapacity_start.min()), int(divvy_dataset.dpcapacity_start.max()), 10), right=False, labels=labels)
divvyTrips['start_station_group'] = pd.cut(divvyTrips.dpcapacity_start, range(0, 120, 10), right=False, labels=labels)
divvyTrips.head()

divvyTrips['end_station_group'] = pd.cut(divvyTrips.dpcapacity_end, range(0, 120, 10), right=False, labels=labels)
divvyTrips.head()

stations_trips = divvyTrips.groupby('start_station_group')['trip_id','gender'].agg(len).dropna()

stations_trips.plot(kind='bar');



## Add Holiday, weekday

In [124]:
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

cal = calendar()
holidays = cal.holidays(start=divvyTrips['starttime'].min(), end=divvyTrips['starttime'].max())
divvyTrips['TypeOfDay'] = divvyTrips['starttime'].dt.date.astype('datetime64').isin(holidays)
divvyTrips.loc[(divvyTrips.DayOfWeek == 5) | (divvyTrips.DayOfWeek == 6) | (divvyTrips.TypeOfDay == True),'TypeOfDay'] = 'Holiday'
divvyTrips.loc[(divvyTrips.TypeOfDay == False),'TypeOfDay'] = 'Weekday'



## Add Seasons to the dataframe

In [137]:
seasons = ['Winter', 'Winter', 'Winter', 'Spring', 'Spring', 'Summer', 'Summer', 'Summer', 'Autumn', 'Autumn', 'Autumn', 'Winter']
month_to_season = dict(zip(range(1,13), seasons))
divvyTrips['season']=divvyTrips.starttime.dt.month.map(month_to_season) 


In [147]:

divvyTrips.to_csv('Social_Economic_Info_By_Zipcode/MergedData/divvy_trips_zipcode_stationdistance.csv')

In [ ]:
columns_to_keep = [
'trip_id'
,'usertype'
,'gender'
,'starttime'
,'stoptime'
,'tripduration'
,'from_station_id'
,'to_station_id'
,'from_station_name'
,'to_station_name'
,'dpcapacity_start'
,'latitude_start'
,'longitude_start'
,'latitude_end'
,'longitude_end'
,'dpcapacity_end'
,'temperature'
,'windchill'
,'dewpoint'
,'humidity'
,'pressure'
,'visibility'
,'wind_speed'
,'precipitation'
,'events'
,'rain'
,'conditions'
,'zipcode_start'
,'zipcode_end'
,'distance_in_meters'
,'Year'
,'Month'
,'Day'
,'DayOfWeek'
,'DayName'
,'DayOfYear'
,'WeekOfYear'
,'Quarter'
,'month_name'
,'months-year'
,'tripInMinutes'
,'start_station_group'
,'end_station_group'
,'geometry_start'
,'geometry_end'
]
divvyTrips[columns_to_keep].head()
divvyTrips.filter(columns_to_keep).head()

## Add Revenue

In [97]:
def trip_cost_15_16_17(trip):
	if trip['usertype']=='Subscriber':
		cost = 0
		if (trip['tripduration'] > 30*60):
			# 30-60 minutes: $3.00
			cost += 1.5
		if (trip['tripduration'] > 60*60):
			# 60-90 minutes: +$4.50
			cost += 3.0
		if (trip['tripduration'] > 90*60):
			# Each additional 30 minutes past 90 minutes: $10.50			
			addl_duration = trip['tripduration'] - 90*60
			cost += 10.50*(addl_duration/(30.0*60))
		if (trip['tripduration'] > 120*60):
			# Each additional 30 minutes past 90 minutes: $10.50			
			addl_duration = trip['tripduration'] - 120*60
			cost += 16.50*(addl_duration/(30.0*60))
		if (trip['tripduration'] > 150*60):
			# Each additional 30 minutes past 90 minutes: $10.50			
			addl_duration = trip['tripduration'] - 150*60
			cost += 22.50*(addl_duration/(30.0*60))
def trip_cost(trip):
	if trip['usertype']=='Subscriber':
		cost = 0
		if (trip['tripduration'] > 30*60) & (trip['tripduration'] <= 60*60):
			# 30-60 minutes: $3.00
			cost += 1.5
		if (trip['tripduration'] > 60*60) & (trip['tripduration'] <= 90*60):
			# 60-90 minutes: +$4.50
			cost += 3.0
		if (trip['tripduration'] > 90*60) & (trip['tripduration'] <= 120*60):
			# Each additional 30 minutes past 90 minutes: $10.50			
			addl_duration = trip['tripduration'] - 90*60
			cost += 10.50*(addl_duration/(30.0*60))
		if (trip['tripduration'] > 120*60) & (trip['tripduration'] <= 150*60):
			# Each additional 30 minutes past 90 minutes: $10.50			
			addl_duration = trip['tripduration'] - 120*60
			cost += 16.50*(addl_duration/(30.0*60))
		if (trip['tripduration'] > 150*60) & (trip['tripduration'] <= 180*60):
			# Each additional 30 minutes past 90 minutes: $10.50			
			addl_duration = trip['tripduration'] - 150*60
			cost += 22.50*(addl_duration/(30.0*60))
		if (trip['tripduration'] > 180*60) & (trip['tripduration'] <= 210*60):
			# Each additional 30 minutes past 90 minutes: $10.50			
			addl_duration = trip['tripduration'] - 180*60
			cost += 28.50*(addl_duration/(30.0*60))
		if (trip['tripduration'] > 210*60) & (trip['tripduration'] <= 240*60):
			# Each additional 30 minutes past 90 minutes: $10.50			
			addl_duration = trip['tripduration'] - 210*60
			cost += 34.50*(addl_duration/(30.0*60))
		if (trip['tripduration'] > 240*60) & (trip['tripduration'] <= 270*60):
			# Each additional 30 minutes past 90 minutes: $10.50			
			addl_duration = trip['tripduration'] - 240*60
			cost += 40.50*(addl_duration/(30.0*60))
		if (trip['tripduration'] > 270*60) & (trip['tripduration'] <= 300*60):
			addl_duration = trip['tripduration'] - 270*60
			cost += 46.50*(addl_duration/(30.0*60))
		if (trip['tripduration'] > 300*60) & (trip['tripduration'] <= 330*60):
			addl_duration = trip['tripduration'] - 300*60
			cost += 52.50*(addl_duration/(30.0*60))
		if (trip['tripduration'] > 330*60) & (trip['tripduration'] <= 360*60):
			addl_duration = trip['tripduration'] - 330*60
			cost += 58.50*(addl_duration/(30.0*60))
		if (trip['tripduration'] > 360*60) & (trip['tripduration'] <= 1440*60):
			# Each additional 6 hours: $76.50			
			addl_duration = trip['tripduration'] - 360*60
			cost += 76.50*(addl_duration/(30.0*60))
		if (trip['tripduration'] > 1440*60):
			# Each additional 24 hours: $1200			
			addl_duration = trip['tripduration'] - 1440*60
			cost += 1200*(addl_duration/(30.0*60))
		return cost
	else:
		cost = 0
		if trip['tripduration'] > 30*60:
			# 30-60 minutes: $2.00
			cost += 2.0
		if trip['tripduration'] > 60*60:
			# 60-90 minutes: $4.00
			cost += 4.0
		if trip['tripduration'] > 90*60:
			# Each additional 30 minutes past 90 minutes: $8.00
			addl_duration = trip['tripduration'] - 90*60
			cost += 8.0*(addl_duration/(30.0*60))
		return cost


def trip_cost_13_14(trip):
	if trip['usertype']=='Subscriber':
		cost = 0
		if trip['tripduration'] > 30*60:
			# 30-60 minutes: $1.50
			cost += 1.5
		if trip['tripduration'] > 60*60:
			# 60-90 minutes: +$3.00
			cost += 3.0
		if trip['tripduration'] > 90*60:
			# Each additional 30 minutes past 90 minutes: $6.00			
			addl_duration = trip['tripduration'] - 90*60
			cost += 6.0*(addl_duration/(30.0*60))
		return cost
	else:
		cost = 0
		if trip['tripduration'] > 30*60:
			# 30-60 minutes: $2.00
			cost += 2.0
		if trip['tripduration'] > 60*60:
			# 60-90 minutes: $4.00
			cost += 4.0
		if trip['tripduration'] > 90*60:
			# Each additional 30 minutes past 90 minutes: $8.00
			addl_duration = trip['tripduration'] - 90*60
			cost += 8.0*(addl_duration/(30.0*60))
		return cost


In [141]:
#divvyTrips.loc[divvyTrips['Year'] == 2013, 'revenue'] = divvyTrips[divvyTrips.Year == 2013].apply(trip_cost_13_14,axis=1)
#divvyTrips.loc[divvyTrips['Year'] == 2014, 'revenue'] = divvyTrips[divvyTrips.Year == 2014].apply(trip_cost_13_14,axis=1)
#divvyTrips.loc[divvyTrips['Year'] == 2015, 'revenue'] = divvyTrips[divvyTrips.Year == 2015].apply(trip_cost_15_16_17,axis=1)
#divvyTrips.loc[divvyTrips['Year'] == 2016, 'revenue'] = divvyTrips[divvyTrips.Year == 2016].apply(trip_cost_15_16_17,axis=1)
#divvyTrips.loc[divvyTrips['Year'] == 2017, 'revenue'] = divvyTrips[divvyTrips.Year == 2017].apply(trip_cost_15_16_17,axis=1)


## Add Geomettry column

In [19]:
# importing libraries
import pandas as pd # Reading csv file 
from shapely.geometry import Point # Shapely for converting latitude/longtitude to geometry
import geopandas as gpd # To create GeodataFrame
# creating a geometry column 
divvyTrips['geometry_start'] = [Point(xy) for xy in zip(divvyTrips['longitude_start'], divvyTrips['latitude_start'])]
divvyTrips['geometry_end'] = [Point(xy) for xy in zip(divvyTrips['longitude_end'], divvyTrips['latitude_end'])]

# Coordinate reference system : WGS84
crs = {'init': 'epsg:4326'}
# Creating a Geographic data frame 
gdf = gpd.GeoDataFrame(divvyTrips, crs=crs, geometry=divvyTrips.geometry_start)
gdf.plot(marker='*', markersize=0.2)

,Unnamed: 0,trip_id,usertype,gender,starttime,stoptime,tripduration,from_station_id,from_station_name,latitude_start,...,Quarter,month_name,months-year,tripInMinutes,start_station_group,end_station_group,revenue,Hour,TypeOfDay,season
0,0,4118,Customer,NaN,2013-06-27 12:11:00,2013-06-27 12:16:00,316,85,Michigan Ave & Oak St,41.900960,...,2,Jun,Jun-2013,5.266667,10 - 19,10 - 19,0.0,12,Weekday,Summer
1,1,4095,Subscriber,Male,2013-06-27 12:06:00,2013-06-27 12:11:00,301,85,Michigan Ave & Oak St,41.900960,...,2,Jun,Jun-2013,5.016667,10 - 19,10 - 19,0.0,12,Weekday,Summer
2,2,4192,Subscriber,Male,2013-06-27 12:15:00,2013-06-27 12:16:00,60,28,Larrabee St & Menomonee St,41.914680,...,2,Jun,Jun-2013,1.000000,10 - 19,10 - 19,0.0,12,Weekday,Summer
3,3,4275,Customer,NaN,2013-06-27 14:44:00,2013-06-27 14:45:00,64,32,Racine Ave & Congress Pkwy,41.874640,...,2,Jun,Jun-2013,1.066667,10 - 19,10 - 19,0.0,14,Weekday,Summer
4,4,4291,Customer,NaN,2013-06-27 14:58:00,2013-06-27 15:05:00,433,32,Racine Ave & Congress Pkwy,41.874640,...,2,Jun,Jun-2013,7.216667,10 - 19,10 - 19,0.0,14,Weekday,Summer
5,5,4263,Subscriber,Male,2013-06-27 14:39:00,2013-06-27 14:40:00,62,61,Wood St & Milwaukee Ave,41.907655,...,2,Jun,Jun-2013,1.033333,10 - 19,10 - 19,0.0,14,Weekday,Summer
6,6,4288,Subscriber,Male,2013-06-27 14:56:00,2013-06-27 14:57:00,66,68,Clinton St & Tilden St,41.875885,...,2,Jun,Jun-2013,1.100000,20 - 29,20 - 29,0.0,14,Weekday,Summer
7,7,4289,Subscriber,Female,2013-06-27 14:57:00,2013-06-27 15:05:00,487,32,Racine Ave & Congress Pkwy,41.874640,...,2,Jun,Jun-2013,8.116667,10 - 19,10 - 19,0.0,14,Weekday,Summer
8,8,4316,Customer,NaN,2013-06-27 15:06:00,2013-06-27 15:09:00,123,19,Loomis St & Taylor St,41.869417,...,2,Jun,Jun-2013,2.050000,10 - 19,10 - 19,0.0,15,Weekday,Summer
9,9,4342,Customer,NaN,2013-06-27 15:13:00,2013-06-27 15:27:00,852,19,Loomis St & Taylor St,41.869417,...,2,Jun,Jun-2013,14.200000,10 - 19,10 - 19,0.0,15,Weekday,Summer
